# Importação dos módulos e dataframe base

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/TCC - Wallynson/perfil_comparecimento_abstencao_2020.csv', encoding = 'latin-1', sep = ';')

In [ ]:
# df_estados_1turno = pd.read_csv('df_estados_1turno_2020.csv')
# df_estados_1turno = df_estados_1turno.rename(columns = {'SG_UF':'estado':'municipio', 'NR_ZONA':'zona'})
# df_estados_1turno = df_estados_1turno.drop(columns = ['Unnamed: 0', 'CD_MUNICIPIO'], axis = 1)

In [ ]:
# df_estados_1turno = pd.read_csv('/content/gdrive/MyDrive/TCC - Wallynson/df_estados_1turno_2020.csv')

In [ ]:
pd.get_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df.shape

In [ ]:
df.isnull().sum() / df.shape[0]

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df = df.drop(columns = ['DT_GERACAO', 'HH_GERACAO', 'CD_MUNICIPIO', 'CD_GENERO',
       'CD_ESTADO_CIVIL',
       'CD_FAIXA_ETARIA', 'CD_GRAU_ESCOLARIDADE'], axis = 1)

# Criação do dataframe do 1º turno

In [ ]:
df_turno_1_2020 = df[(df['NR_TURNO'] == 1)]

In [ ]:
df_turno_1_2020.shape

## Verificação da coluna DS_GENERO

In [ ]:
df_turno_1_2020.DS_GENERO.value_counts()

In [ ]:
#verificação da quantidade percentual de valores 'NÃO INFORMADO' no conjunto
print('Os valores \'NÃO INFORMADO\' representam {}% do conjunto'.format(round(df_turno_1_2020.DS_GENERO.value_counts()[2] / len(df_turno_1_2020), 4)).replace('.', ','))

Como há 22068 (0,005% do conjunto) linhas em que o sexo foi registrado como NÃO INFORMADO, optei por excluí-las.

## Verificação da coluna DS_GRAU_ESCOLARIDADE

In [ ]:
df_turno_1_2020.DS_GRAU_ESCOLARIDADE.value_counts()

In [ ]:
#verificação da quantidade percentual de valores 'NÃO INFORMADO' no conjunto
print('Os valores \'NÃO INFORMADO\' representam {}% do conjunto'.format(round(df_turno_1_2020.DS_GRAU_ESCOLARIDADE.value_counts()[8] / len(df_turno_1_2020), 4)).replace('.', ','))

Como há 16439 (0,0038% do conjunto) linhas em que o grau de escolaridade foi registrado como NÃO INFORMADO, optei por excluí-las.

## Verificação da coluna DS_FAIXA_ETARIA

In [ ]:
df_turno_1_2020.DS_FAIXA_ETARIA.value_counts()

In [ ]:
#verificação da quantidade percentual de valores 'NÃO INFORMADO' no conjunto
print('Os valores \'Inválido\' representam {}% do conjunto'.format(round(df_turno_1_2020.DS_FAIXA_ETARIA.value_counts()[20] / len(df_turno_1_2020), 4)).replace('.', ','))

Como há 52839 (0,0121% do conjunto) linhas em que a faixa etária foi registrada como INVÁLIDO, optei por excluí-las.

## Verificação da coluna DS_ESTADO_CIVIL

In [ ]:
df_turno_1_2020.DS_ESTADO_CIVIL.value_counts()

In [ ]:
#verificação da quantidade percentual de valores 'NÃO INFORMADO' no conjunto
print('Os valores \'NÃO INFORMADO\' representam {}% do conjunto'.format(round(df_turno_1_2020.DS_ESTADO_CIVIL.value_counts()[5] / len(df_turno_1_2020), 4)).replace('.', ','))

Como há 33780 (0,0077% do conjunto) linhas em que o estado civil foi registrada como NÃO INFORMADO, optei por excluí-las.

## Exclusão dos valores 'NÃO INFORMADO', 'Inválido' e 'ZZ' (que são os dados do exterior).

In [ ]:
df_turno_1_2020 = df_turno_1_2020.drop(df_turno_1_2020.loc[df_turno_1_2020['DS_GENERO'] == 'NÃO INFORMADO'].index)

In [ ]:
df_turno_1_2020 = df_turno_1_2020.drop(df_turno_1_2020.loc[df_turno_1_2020['DS_GRAU_ESCOLARIDADE'] == 'NÃO INFORMADO'].index)

In [ ]:
df_turno_1_2020 = df_turno_1_2020.drop(df_turno_1_2020.loc[df_turno_1_2020['DS_FAIXA_ETARIA'] == 'Inválido'].index)

In [ ]:
df_turno_1_2020 = df_turno_1_2020.drop(df_turno_1_2020.loc[df_turno_1_2020['DS_ESTADO_CIVIL'] == 'NÃO INFORMADO'].index)

In [ ]:
df_turno_1_2020 = df_turno_1_2020.drop(df_turno_1_2020.loc[df_turno_1_2020['SG_UF'] == 'ZZ'].index)

## Ordenação do dataframe por estado e município

In [ ]:
df_turno_1_2020 = df_turno_1_2020.sort_values(by = ['SG_UF'], ascending = True)

# Criação do dataset por estado

In [ ]:
data = {'SG_UF': df_turno_1_2020.SG_UF.unique()}
df_estados_1turno = pd.DataFrame(data)

## Eleitores aptos

In [ ]:
df_estados_1turno = df_estados_1turno.merge(df_turno_1_2020.groupby(['SG_UF'])['QT_APTOS'].sum().reset_index(), on=['SG_UF'])
df_estados_1turno = df_estados_1turno.rename(columns={'QT_APTOS': 'aptos'})

## Comparecimento do eleitorado

In [ ]:
df_estados_1turno = df_estados_1turno.merge(df_turno_1_2020.groupby(['SG_UF'])['QT_COMPARECIMENTO'].sum().reset_index(), on=['SG_UF'])
df_estados_1turno = df_estados_1turno.rename(columns={'QT_COMPARECIMENTO': 'comparecimento'})

### Comparecimento percentual do eleitorado

In [ ]:
df_estados_1turno['comparecimento_percentual(%)'] = (df_estados_1turno['comparecimento'] / df_estados_1turno['aptos']) * 100
df_estados_1turno['comparecimento_percentual(%)'] = df_estados_1turno['comparecimento_percentual(%)'].round(2)

## Abstenção do eleitorado

In [ ]:
df_estados_1turno = df_estados_1turno.merge(df_turno_1_2020.groupby(['SG_UF'])['QT_ABSTENCAO'].sum().reset_index(), on=['SG_UF'])
df_estados_1turno = df_estados_1turno.rename(columns={'QT_ABSTENCAO': 'abstencao'})

### Abstencao percentual do eleitorado

In [ ]:
df_estados_1turno['abstencao_percentual(%)'] = (df_estados_1turno['abstencao'] / df_estados_1turno['aptos']) * 100
df_estados_1turno['abstencao_percentual(%)'] = df_estados_1turno['abstencao_percentual(%)'].round(2)

## Eleitorado com deficiência

In [ ]:
df_estados_1turno = df_estados_1turno.merge(df_turno_1_2020.groupby(['SG_UF'])['QT_COMPARECIMENTO_DEFICIENCIA'].sum().reset_index(), on=['SG_UF'])
df_estados_1turno = df_estados_1turno.rename(columns={'QT_COMPARECIMENTO_DEFICIENCIA': 'comparecimento_deficiencia'})

df_estados_1turno = df_estados_1turno.merge(df_turno_1_2020.groupby(['SG_UF'])['QT_ABSTENCAO_DEFICIENCIA'].sum().reset_index(), on=['SG_UF'])
df_estados_1turno = df_estados_1turno.rename(columns={'QT_ABSTENCAO_DEFICIENCIA': 'abstencao_deficiencia'})

df_estados_1turno['eleitores_deficiencia'] = df_estados_1turno['comparecimento_deficiencia'] + df_estados_1turno['abstencao_deficiencia']

### Eleitorado percentual com deficiência

In [ ]:
df_estados_1turno['eleitores_deficiencia_percentual(%)'] = (df_estados_1turno['eleitores_deficiencia'] / df_estados_1turno['aptos']) * 100
df_estados_1turno['eleitores_deficiencia_percentual(%)'] = df_estados_1turno['eleitores_deficiencia_percentual(%)'].round(2)

## Eleitorado feminino

In [ ]:
eleitorado_feminino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'FEMININO'].groupby(['SG_UF'])['QT_APTOS'].sum().reset_index()

df_estados_1turno = df_estados_1turno.merge(eleitorado_feminino, on=['SG_UF'], how='left')
df_estados_1turno['eleitorado_feminino'] = df_estados_1turno['QT_APTOS'].fillna(0).astype(int)
df_estados_1turno = df_estados_1turno.drop(columns=['QT_APTOS'])

del eleitorado_feminino

### Eleitorado percentual feminino

In [ ]:
df_estados_1turno['eleitorado_feminino_percentual(%)'] = ((df_estados_1turno['eleitorado_feminino'] / df_estados_1turno['aptos']) * 100)
df_estados_1turno['eleitorado_feminino_percentual(%)'] = df_estados_1turno['eleitorado_feminino_percentual(%)'].round(2)

## Eleitorado masculino

In [ ]:
eleitorado_masculino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'MASCULINO'].groupby(['SG_UF'])['QT_APTOS'].sum().reset_index()

df_estados_1turno = df_estados_1turno.merge(eleitorado_masculino, on=['SG_UF'], how='left')
df_estados_1turno['eleitorado_masculino'] = df_estados_1turno['QT_APTOS'].fillna(0).astype(int)
df_estados_1turno = df_estados_1turno.drop(columns=['QT_APTOS'])

del eleitorado_masculino

### Eleitorado percentual masculino

In [ ]:
df_estados_1turno['eleitorado_masculino_percentual(%)'] = ((df_estados_1turno['eleitorado_masculino'] / df_estados_1turno['aptos']) * 100)
df_estados_1turno['eleitorado_masculino_percentual(%)'] = df_estados_1turno['eleitorado_masculino_percentual(%)'].round(2)

## Escolaridade do eleitorado


In [ ]:
escolaridade_pivot = df_turno_1_2020.pivot_table(index=['SG_UF'], columns='DS_GRAU_ESCOLARIDADE', values='QT_APTOS', aggfunc='sum', fill_value=0)

# Combina a tabela pivot com o DataFrame df_estados_1turno
df_estados_1turno = pd.merge(df_estados_1turno, escolaridade_pivot, on=['SG_UF'])

# Remove os prefixos dos nomes das colunas gerados pela tabela pivot
df_estados_1turno.columns = df_estados_1turno.columns.str.replace('DS_GRAU_ESCOLARIDADE_', '')
df_estados_1turno = df_estados_1turno.rename(columns={'ANALFABETO':'analfabeto', 'ENSINO FUNDAMENTAL COMPLETO':'fundamental_completo',
  'ENSINO FUNDAMENTAL INCOMPLETO':'fundamental_incompleto', 'ENSINO MÉDIO COMPLETO':'medio_completo', 'ENSINO MÉDIO INCOMPLETO':'medio_incompleto',
  'LÊ E ESCREVE':'le_escreve', 'SUPERIOR COMPLETO':'superior_completo', 'SUPERIOR INCOMPLETO':'superior_incompleto'})

del escolaridade_pivot

In [ ]:
df_estados_1turno = df_estados_1turno[['SG_UF', 'aptos', 'comparecimento',
       'comparecimento_percentual(%)', 'abstencao', 'abstencao_percentual(%)',
       'comparecimento_deficiencia', 'abstencao_deficiencia',
       'eleitores_deficiencia', 'eleitores_deficiencia_percentual(%)',
       'eleitorado_feminino', 'eleitorado_feminino_percentual(%)',
       'eleitorado_masculino', 'eleitorado_masculino_percentual(%)',
       'analfabeto', 'le_escreve', 'fundamental_incompleto', 'fundamental_completo',
       'medio_incompleto', 'medio_completo', 'superior_incompleto','superior_completo',]]

### Escolaridade percentual do eleitorado

In [ ]:
def define_escolaridade_percentual(row, coluna):
    percentual = (row[coluna] / row['aptos']) * 100
    return round(percentual, 2)

In [ ]:
# Lista das colunas de escolaridade
colunas_escolaridade = ['analfabeto', 'le_escreve', 'fundamental_incompleto', 'fundamental_completo',
                        'medio_incompleto', 'medio_completo', 'superior_incompleto', 'superior_completo']

# Aplica a função define_escolaridade_percentual() em todas as colunas relevantes
for coluna in colunas_escolaridade:
    df_estados_1turno[coluna + '_percentual(%)'] = df_estados_1turno.apply(
        lambda row: define_escolaridade_percentual(row, coluna), axis=1)

del colunas_escolaridade

### Escolaridade do eleitorado masculino

In [ ]:
# Filtra os dados apenas para o gênero masculino
df_turno_1_2020_masculino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'MASCULINO']

# Realiza a agregação dos dados por município e grau de escolaridade
agregado_masculino = df_turno_1_2020_masculino.groupby(['SG_UF', 'DS_GRAU_ESCOLARIDADE'])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = agregado_masculino.pivot_table(index=['SG_UF'], columns='DS_GRAU_ESCOLARIDADE', values='QT_APTOS').reset_index()

# Junta os dados no dataframe df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')
df_estados_1turno = df_estados_1turno.rename(columns={'ANALFABETO':'analfabeto_masculino',
  'ENSINO FUNDAMENTAL COMPLETO':'fundamental_completo_masculino', 'ENSINO FUNDAMENTAL INCOMPLETO':'fundamental_incompleto_masculino',
  'ENSINO MÉDIO COMPLETO':'medio_completo_masculino', 'ENSINO MÉDIO INCOMPLETO':'medio_incompleto_masculino',
  'LÊ E ESCREVE':'le_escreve_masculino', 'SUPERIOR COMPLETO':'superior_completo_masculino',
  'SUPERIOR INCOMPLETO':'superior_incompleto_masculino'})

del df_turno_1_2020_masculino, agregado_masculino, pivot_masculino

In [ ]:
colunas_escolaridade_masculina = ['analfabeto_masculino', 'le_escreve_masculino', 'fundamental_incompleto_masculino',
                                 'fundamental_completo_masculino', 'medio_incompleto_masculino',
                                 'medio_completo_masculino', 'superior_incompleto_masculino',
                                 'superior_completo_masculino']
for coluna in colunas_escolaridade_masculina:
  df_estados_1turno[coluna] = df_estados_1turno[coluna].fillna(0)
  df_estados_1turno[coluna] = df_estados_1turno[coluna].astype(int)

del colunas_escolaridade_masculina

In [ ]:
df_estados_1turno = df_estados_1turno[['SG_UF', 'aptos', 'comparecimento',
       'comparecimento_percentual(%)', 'abstencao', 'abstencao_percentual(%)',
       'comparecimento_deficiencia', 'abstencao_deficiencia',
       'eleitores_deficiencia', 'eleitores_deficiencia_percentual(%)',
       'eleitorado_feminino', 'eleitorado_feminino_percentual(%)',
       'eleitorado_masculino', 'eleitorado_masculino_percentual(%)',
       'analfabeto', 'le_escreve', 'fundamental_incompleto',
       'fundamental_completo', 'medio_incompleto', 'medio_completo',
       'superior_incompleto', 'superior_completo', 'analfabeto_percentual(%)',
       'le_escreve_percentual(%)', 'fundamental_incompleto_percentual(%)',
       'fundamental_completo_percentual(%)', 'medio_incompleto_percentual(%)',
       'medio_completo_percentual(%)', 'superior_incompleto_percentual(%)',
       'superior_completo_percentual(%)', 'analfabeto_masculino',
       'le_escreve_masculino', 'fundamental_incompleto_masculino',
       'fundamental_completo_masculino', 'medio_incompleto_masculino',
       'medio_completo_masculino', 'superior_incompleto_masculino',
       'superior_completo_masculino']]

#### Escolaridade percentual do eleitorado masculino

In [ ]:
def define_escolaridade_percentual_masculina(row, coluna):
    if row[coluna.replace('_masculino', '')] != 0:
        return round((row[coluna] / row[coluna.replace('_masculino', '')]) * 100, 2)
    return 0.0

In [ ]:
colunas_escolaridade_masculina = ['analfabeto_masculino', 'le_escreve_masculino', 'fundamental_incompleto_masculino',
                                 'fundamental_completo_masculino', 'medio_incompleto_masculino',
                                 'medio_completo_masculino', 'superior_incompleto_masculino',
                                 'superior_completo_masculino']

for coluna in colunas_escolaridade_masculina:
    coluna_percentual = coluna + '_percentual(%)'
    df_estados_1turno[coluna_percentual] = df_estados_1turno.apply(define_escolaridade_percentual_masculina,
                                                                         args=(coluna,),
                                                                         axis=1)

del colunas_escolaridade_masculina

### Escolaridade do eleitorado feminino

In [ ]:
# Filtra os dados apenas para o gênero feminino
df_turno_1_2020_feminino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'FEMININO']

# Realiza a agregação dos dados por município e grau de escolaridade
agregado_feminino = df_turno_1_2020_feminino.groupby(['SG_UF', 'DS_GRAU_ESCOLARIDADE'])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = agregado_feminino.pivot_table(index=['SG_UF'], columns='DS_GRAU_ESCOLARIDADE', values='QT_APTOS').reset_index()

# Junta os dados no dataframe df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')
df_estados_1turno = df_estados_1turno.rename(columns={'ANALFABETO':'analfabeto_feminino',
  'ENSINO FUNDAMENTAL COMPLETO':'fundamental_completo_feminino', 'ENSINO FUNDAMENTAL INCOMPLETO':'fundamental_incompleto_feminino',
  'ENSINO MÉDIO COMPLETO':'medio_completo_feminino', 'ENSINO MÉDIO INCOMPLETO':'medio_incompleto_feminino',
  'LÊ E ESCREVE':'le_escreve_feminino', 'SUPERIOR COMPLETO':'superior_completo_feminino',
  'SUPERIOR INCOMPLETO':'superior_incompleto_feminino'})

del df_turno_1_2020_feminino, agregado_feminino, pivot_feminino

In [ ]:
colunas_escolaridade_feminina = ['analfabeto_feminino', 'le_escreve_feminino', 'fundamental_incompleto_feminino',
                                 'fundamental_completo_feminino', 'medio_incompleto_feminino',
                                 'medio_completo_feminino', 'superior_incompleto_feminino',
                                 'superior_completo_feminino']
for coluna in colunas_escolaridade_feminina:
  df_estados_1turno[coluna] = df_estados_1turno[coluna].fillna(0)
  df_estados_1turno[coluna] = df_estados_1turno[coluna].astype(int)

In [ ]:
df_estados_1turno = df_estados_1turno[['SG_UF', 'aptos', 'comparecimento',
       'comparecimento_percentual(%)', 'abstencao', 'abstencao_percentual(%)',
       'comparecimento_deficiencia', 'abstencao_deficiencia',
       'eleitores_deficiencia', 'eleitores_deficiencia_percentual(%)',
       'eleitorado_feminino', 'eleitorado_feminino_percentual(%)',
       'eleitorado_masculino', 'eleitorado_masculino_percentual(%)',
       'analfabeto', 'le_escreve', 'fundamental_incompleto',
       'fundamental_completo', 'medio_incompleto', 'medio_completo',
       'superior_incompleto', 'superior_completo', 'analfabeto_percentual(%)',
       'le_escreve_percentual(%)', 'fundamental_incompleto_percentual(%)',
       'fundamental_completo_percentual(%)', 'medio_incompleto_percentual(%)',
       'medio_completo_percentual(%)', 'superior_incompleto_percentual(%)',
       'superior_completo_percentual(%)', 'analfabeto_masculino',
       'le_escreve_masculino', 'fundamental_incompleto_masculino',
       'fundamental_completo_masculino', 'medio_incompleto_masculino',
       'medio_completo_masculino', 'superior_incompleto_masculino',
       'superior_completo_masculino', 'analfabeto_masculino_percentual(%)',
       'le_escreve_masculino_percentual(%)',
       'fundamental_incompleto_masculino_percentual(%)',
       'fundamental_completo_masculino_percentual(%)',
       'medio_incompleto_masculino_percentual(%)',
       'medio_completo_masculino_percentual(%)',
       'superior_incompleto_masculino_percentual(%)',
       'superior_completo_masculino_percentual(%)', 'analfabeto_feminino',
       'le_escreve_feminino', 'fundamental_incompleto_feminino',
       'fundamental_completo_feminino', 'medio_incompleto_feminino',
       'medio_completo_feminino', 'superior_incompleto_feminino',
       'superior_completo_feminino']]

#### Escolaridade percentual do eleitorado feminino

In [ ]:
def define_escolaridade_percentual_feminina(row, coluna):
    if row[coluna.replace('_feminino', '')] != 0:
        return round((row[coluna] / row[coluna.replace('_feminino', '')]) * 100, 2)
    return 0.0

In [ ]:
colunas_escolaridade_feminina = ['analfabeto_feminino', 'le_escreve_feminino', 'fundamental_incompleto_feminino',
                                 'fundamental_completo_feminino', 'medio_incompleto_feminino',
                                 'medio_completo_feminino', 'superior_incompleto_feminino',
                                 'superior_completo_feminino']

for coluna in colunas_escolaridade_feminina:
    coluna_percentual = coluna + '_percentual(%)'
    df_estados_1turno[coluna_percentual] = df_estados_1turno.apply(define_escolaridade_percentual_feminina,
                                                                         args=(coluna,),
                                                                         axis=1)

## Faixa etária do eleitorado

In [ ]:
# Calcular as somas das faixas etárias por município
faixa_etaria_sum = df_turno_1_2020.groupby(['SG_UF', 'DS_FAIXA_ETARIA'])['QT_APTOS'].sum().reset_index()

# Pivotar a tabela para obter as colunas desejadas
df_pivot = faixa_etaria_sum.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS', fill_value=0).reset_index()

# Juntar os resultados ao DataFrame df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(df_pivot, how='left', left_on=['SG_UF'], right_on=['SG_UF'])

In [ ]:
df_estados_1turno = df_estados_1turno[['SG_UF', 'aptos', 'comparecimento', 'comparecimento_percentual(%)',
       'abstencao', 'abstencao_percentual(%)', 'comparecimento_deficiencia',
       'abstencao_deficiencia', 'eleitores_deficiencia',
       'eleitores_deficiencia_percentual(%)', 'eleitorado_feminino',
       'eleitorado_feminino_percentual(%)', 'eleitorado_masculino',
       'eleitorado_masculino_percentual(%)', 'analfabeto', 'le_escreve',
       'fundamental_incompleto', 'fundamental_completo', 'medio_incompleto',
       'medio_completo', 'superior_incompleto', 'superior_completo',
       'analfabeto_percentual(%)', 'le_escreve_percentual(%)',
       'fundamental_incompleto_percentual(%)',
       'fundamental_completo_percentual(%)', 'medio_incompleto_percentual(%)',
       'medio_completo_percentual(%)', 'superior_incompleto_percentual(%)',
       'superior_completo_percentual(%)', 'analfabeto_masculino',
       'le_escreve_masculino', 'fundamental_incompleto_masculino',
       'fundamental_completo_masculino', 'medio_incompleto_masculino',
       'medio_completo_masculino', 'superior_incompleto_masculino',
       'superior_completo_masculino', 'analfabeto_masculino_percentual(%)',
       'le_escreve_masculino_percentual(%)',
       'fundamental_incompleto_masculino_percentual(%)',
       'fundamental_completo_masculino_percentual(%)',
       'medio_incompleto_masculino_percentual(%)',
       'medio_completo_masculino_percentual(%)',
       'superior_incompleto_masculino_percentual(%)',
       'superior_completo_masculino_percentual(%)', 'analfabeto_feminino',
       'le_escreve_feminino', 'fundamental_incompleto_feminino',
       'fundamental_completo_feminino', 'medio_incompleto_feminino',
       'medio_completo_feminino', 'superior_incompleto_feminino',
       'superior_completo_feminino', 'analfabeto_feminino_percentual(%)',
       'le_escreve_feminino_percentual(%)',
       'fundamental_incompleto_feminino_percentual(%)',
       'fundamental_completo_feminino_percentual(%)',
       'medio_incompleto_feminino_percentual(%)',
       'medio_completo_feminino_percentual(%)',
       'superior_incompleto_feminino_percentual(%)',
       'superior_completo_feminino_percentual(%)',
       '16 anos', '17 anos', '18 anos', '19 anos', '20 anos', '21 a 24 anos',
       '25 a 29 anos', '30 a 34 anos', '35 a 39 anos', '40 a 44 anos',
       '45 a 49 anos', '50 a 54 anos', '55 a 59 anos', '60 a 64 anos',
       '65 a 69 anos', '70 a 74 anos', '75 a 79 anos', '80 a 84 anos',
       '85 a 89 anos', '90 a 94 anos', '95 a 99 anos', '100 anos ou mais']]

In [ ]:
df_estados_1turno = df_estados_1turno.rename(columns={'100 anos ou mais':'100_anos',
       '16 anos':'16_anos', '17 anos':'17_anos', '18 anos':'18_anos',
       '19 anos':'19_anos', '20 anos':'20_anos', '21 a 24 anos':'21_24_anos',
       '25 a 29 anos':'25_29_anos', '30 a 34 anos':'30_34_anos', '35 a 39 anos':'35_39_anos',
       '40 a 44 anos':'40_44_anos', '45 a 49 anos':'45_49_anos',
       '50 a 54 anos':'50_54_anos', '55 a 59 anos':'55_59_anos',
       '60 a 64 anos':'60_64_anos', '65 a 69 anos':'65_69_anos',
       '70 a 74 anos':'70_74_anos', '75 a 79 anos':'75_79_anos',
       '80 a 84 anos':'80_84_anos', '85 a 89 anos':'85_89_anos',
       '90 a 94 anos':'90_94_anos', '95 a 99 anos':'95_99_anos'})

### Faixa etária percentual do eleitorado

In [ ]:
faixa_etaria_colunas = {
    '16_anos': '16_anos_percentual(%)',
    '17_anos': '17_anos_percentual(%)',
    '18_anos': '18_anos_percentual(%)',
    '19_anos': '19_anos_percentual(%)',
    '20_anos': '20_anos_percentual(%)',
    '21_24_anos': '21_24_anos_percentual(%)',
    '25_29_anos': '25_29_anos_percentual(%)',
    '30_34_anos': '30_34_anos_percentual(%)',
    '35_39_anos': '35_39_anos_percentual(%)',
    '40_44_anos': '40_44_anos_percentual(%)',
    '45_49_anos': '45_49_anos_percentual(%)',
    '50_54_anos': '50_54_anos_percentual(%)',
    '55_59_anos': '55_59_anos_percentual(%)',
    '60_64_anos': '60_64_anos_percentual(%)',
    '65_69_anos': '65_69_anos_percentual(%)',
    '70_74_anos': '70_74_anos_percentual(%)',
    '75_79_anos': '75_79_anos_percentual(%)',
    '80_84_anos': '80_84_anos_percentual(%)',
    '85_89_anos': '85_89_anos_percentual(%)',
    '90_94_anos': '90_94_anos_percentual(%)',
    '95_99_anos': '95_99_anos_percentual(%)',
    '100_anos': '100_anos_percentual(%)'
}

for faixa_etaria, coluna_percentual in faixa_etaria_colunas.items():
  df_estados_1turno[coluna_percentual] = (df_estados_1turno[faixa_etaria] / df_estados_1turno['aptos'] * 100).round(2)


### Faixa etária do eleitorado masculino

In [ ]:
faixa_etaria_masculino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'MASCULINO'].groupby(['SG_UF', 'DS_FAIXA_ETARIA'])['QT_APTOS'].sum().reset_index()
# Pivotar a tabela para obter as colunas desejadas
df_pivot = faixa_etaria_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS', fill_value=0).reset_index()

# Juntar os resultados ao DataFrame df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(df_pivot, how='left', left_on=['SG_UF'], right_on=['SG_UF'])
# df_estados_1turno = df_estados_1turno.sort_values(by=['SG_UF'])


In [ ]:
df_estados_1turno = df_estados_1turno.rename(columns={
       '16 anos':'16_anos_masculino', '17 anos':'17_anos_masculino', '18 anos':'18_anos_masculino',
       '19 anos':'19_anos_masculino', '20 anos':'20_anos_masculino', '21 a 24 anos':'21_24_anos_masculino',
       '25 a 29 anos':'25_29_anos_masculino', '30 a 34 anos':'30_34_anos_masculino', '35 a 39 anos':'35_39_anos_masculino',
       '40 a 44 anos':'40_44_anos_masculino', '45 a 49 anos':'45_49_anos_masculino',
       '50 a 54 anos':'50_54_anos_masculino', '55 a 59 anos':'55_59_anos_masculino',
       '60 a 64 anos':'60_64_anos_masculino', '65 a 69 anos':'65_69_anos_masculino',
       '70 a 74 anos':'70_74_anos_masculino', '75 a 79 anos':'75_79_anos_masculino',
       '80 a 84 anos':'80_84_anos_masculino', '85 a 89 anos':'85_89_anos_masculino',
       '90 a 94 anos':'90_94_anos_masculino', '95 a 99 anos':'95_99_anos_masculino',
       '100 anos ou mais':'100_anos_masculino',})

In [ ]:
faixa_etaria_masculina_colunas = {
    '16_anos_masculino_percentual(%)': ['16_anos_masculino', '16_anos'],
    '17_anos_masculino_percentual(%)': ['17_anos_masculino', '17_anos'],
    '18_anos_masculino_percentual(%)': ['18_anos_masculino', '18_anos'],
    '19_anos_masculino_percentual(%)': ['19_anos_masculino', '19_anos'],
    '20_anos_masculino_percentual(%)': ['20_anos_masculino', '20_anos'],
    '21_24_anos_masculino_percentual(%)': ['21_24_anos_masculino', '21_24_anos'],
    '25_29_anos_masculino_percentual(%)': ['25_29_anos_masculino', '25_29_anos'],
    '30_34_anos_masculino_percentual(%)': ['30_34_anos_masculino', '30_34_anos'],
    '35_39_anos_masculino_percentual(%)': ['35_39_anos_masculino', '35_39_anos'],
    '40_44_anos_masculino_percentual(%)': ['40_44_anos_masculino', '40_44_anos'],
    '45_49_anos_masculino_percentual(%)': ['45_49_anos_masculino', '45_49_anos'],
    '50_54_anos_masculino_percentual(%)': ['50_54_anos_masculino', '50_54_anos'],
    '55_59_anos_masculino_percentual(%)': ['55_59_anos_masculino', '55_59_anos'],
    '60_64_anos_masculino_percentual(%)': ['60_64_anos_masculino', '60_64_anos'],
    '65_69_anos_masculino_percentual(%)': ['65_69_anos_masculino', '65_69_anos'],
    '70_74_anos_masculino_percentual(%)': ['70_74_anos_masculino', '70_74_anos'],
    '75_79_anos_masculino_percentual(%)': ['75_79_anos_masculino', '75_79_anos'],
    '80_84_anos_masculino_percentual(%)': ['80_84_anos_masculino', '80_84_anos'],
    '85_89_anos_masculino_percentual(%)': ['85_89_anos_masculino', '85_89_anos'],
    '90_94_anos_masculino_percentual(%)': ['90_94_anos_masculino', '90_94_anos'],
    '95_99_anos_masculino_percentual(%)': ['95_99_anos_masculino', '95_99_anos'],
    '100_anos_masculino_percentual(%)'  : ['100_anos_masculino', '100_anos']
}

for coluna_percentual, faixa_etaria in faixa_etaria_masculina_colunas.items():
  df_estados_1turno[coluna_percentual] = (df_estados_1turno[faixa_etaria[0]] / df_estados_1turno[faixa_etaria[1]] * 100).round(2)

### Faixa etária do eleitorado feminino

In [ ]:
faixa_etaria_feminino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'FEMININO'].groupby(['SG_UF', 'DS_FAIXA_ETARIA'])['QT_APTOS'].sum().reset_index()
# Pivotar a tabela para obter as colunas desejadas
df_pivot = faixa_etaria_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS', fill_value=0).reset_index()

# Juntar os resultados ao DataFrame df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(df_pivot, how='left', left_on=['SG_UF'], right_on=['SG_UF'])
# df_estados_1turno = df_estados_1turno.sort_values(by=['SG_UF'])

In [ ]:
df_estados_1turno = df_estados_1turno.rename(columns={
       '16 anos':'16_anos_feminino', '17 anos':'17_anos_feminino', '18 anos':'18_anos_feminino',
       '19 anos':'19_anos_feminino', '20 anos':'20_anos_feminino', '21 a 24 anos':'21_24_anos_feminino',
       '25 a 29 anos':'25_29_anos_feminino', '30 a 34 anos':'30_34_anos_feminino', '35 a 39 anos':'35_39_anos_feminino',
       '40 a 44 anos':'40_44_anos_feminino', '45 a 49 anos':'45_49_anos_feminino',
       '50 a 54 anos':'50_54_anos_feminino', '55 a 59 anos':'55_59_anos_feminino',
       '60 a 64 anos':'60_64_anos_feminino', '65 a 69 anos':'65_69_anos_feminino',
       '70 a 74 anos':'70_74_anos_feminino', '75 a 79 anos':'75_79_anos_feminino',
       '80 a 84 anos':'80_84_anos_feminino', '85 a 89 anos':'85_89_anos_feminino',
       '90 a 94 anos':'90_94_anos_feminino', '95 a 99 anos':'95_99_anos_feminino',
       '100 anos ou mais':'100_anos_feminino',})

In [ ]:
faixa_etaria_feminina_colunas = {
    '16_anos_feminino_percentual(%)': ['16_anos_feminino', '16_anos'],
    '17_anos_feminino_percentual(%)': ['17_anos_feminino', '17_anos'],
    '18_anos_feminino_percentual(%)': ['18_anos_feminino', '18_anos'],
    '19_anos_feminino_percentual(%)': ['19_anos_feminino', '19_anos'],
    '20_anos_feminino_percentual(%)': ['20_anos_feminino', '20_anos'],
    '21_24_anos_feminino_percentual(%)': ['21_24_anos_feminino', '21_24_anos'],
    '25_29_anos_feminino_percentual(%)': ['25_29_anos_feminino', '25_29_anos'],
    '30_34_anos_feminino_percentual(%)': ['30_34_anos_feminino', '30_34_anos'],
    '35_39_anos_feminino_percentual(%)': ['35_39_anos_feminino', '35_39_anos'],
    '40_44_anos_feminino_percentual(%)': ['40_44_anos_feminino', '40_44_anos'],
    '45_49_anos_feminino_percentual(%)': ['45_49_anos_feminino', '45_49_anos'],
    '50_54_anos_feminino_percentual(%)': ['50_54_anos_feminino', '50_54_anos'],
    '55_59_anos_feminino_percentual(%)': ['55_59_anos_feminino', '55_59_anos'],
    '60_64_anos_feminino_percentual(%)': ['60_64_anos_feminino', '60_64_anos'],
    '65_69_anos_feminino_percentual(%)': ['65_69_anos_feminino', '65_69_anos'],
    '70_74_anos_feminino_percentual(%)': ['70_74_anos_feminino', '70_74_anos'],
    '75_79_anos_feminino_percentual(%)': ['75_79_anos_feminino', '75_79_anos'],
    '80_84_anos_feminino_percentual(%)': ['80_84_anos_feminino', '80_84_anos'],
    '85_89_anos_feminino_percentual(%)': ['85_89_anos_feminino', '85_89_anos'],
    '90_94_anos_feminino_percentual(%)': ['90_94_anos_feminino', '90_94_anos'],
    '95_99_anos_feminino_percentual(%)': ['95_99_anos_feminino', '95_99_anos'],
    '100_anos_feminino_percentual(%)'  : ['100_anos_feminino', '100_anos']
}

for coluna_percentual, faixa_etaria in faixa_etaria_feminina_colunas.items():
  df_estados_1turno[coluna_percentual] = (df_estados_1turno[faixa_etaria[0]] / df_estados_1turno[faixa_etaria[1]] * 100).round(2)

## Eleitorado facultativo

In [ ]:
def eleitorado_facultativo_estados_1turno():
  return int(df_estados_1turno['16_anos'].values[i]
  + df_estados_1turno['17_anos'].values[i]
  + df_estados_1turno['65_69_anos'].values[i]
  + df_estados_1turno['70_74_anos'].values[i]
  + df_estados_1turno['75_79_anos'].values[i]
  + df_estados_1turno['80_84_anos'].values[i]
  + df_estados_1turno['85_89_anos'].values[i]
  + df_estados_1turno['90_94_anos'].values[i]
  + df_estados_1turno['95_99_anos'].values[i]
  + df_estados_1turno['100_anos'].values[i])

In [ ]:
df_estados_1turno['eleitorado_facultativo'] = 0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo'][i] = eleitorado_facultativo_estados_1turno()

In [ ]:
def eleitorado_facultativo_estados_1turno_percentual():
  return round(((df_estados_1turno['eleitorado_facultativo'][i] / df_estados_1turno['aptos'][i]) * 100), 2)

In [ ]:
df_estados_1turno['eleitorado_facultativo_percentual(%)'] = 0.0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo_percentual(%)'][i] = eleitorado_facultativo_estados_1turno_percentual()

### Eleitorado facultativo masculino

In [ ]:
def eleitorado_facultativo_masculino_estados_1turno():
  return int(df_estados_1turno['16_anos_masculino'].values[i]
  + df_estados_1turno['17_anos_masculino'].values[i]
  + df_estados_1turno['65_69_anos_masculino'].values[i]
  + df_estados_1turno['70_74_anos_masculino'].values[i]
  + df_estados_1turno['75_79_anos_masculino'].values[i]
  + df_estados_1turno['80_84_anos_masculino'].values[i]
  + df_estados_1turno['85_89_anos_masculino'].values[i]
  + df_estados_1turno['90_94_anos_masculino'].values[i]
  + df_estados_1turno['95_99_anos_masculino'].values[i]
  + df_estados_1turno['100_anos_masculino'].values[i])

In [ ]:
df_estados_1turno['eleitorado_facultativo_masculino'] = 0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo_masculino'][i] = eleitorado_facultativo_masculino_estados_1turno()

In [ ]:
def eleitorado_facultativo_masculino_estados_1turno_percentual():
  return round(((df_estados_1turno['eleitorado_facultativo_masculino'][i] / df_estados_1turno['aptos'][i]) * 100), 2)

In [ ]:
df_estados_1turno['eleitorado_facultativo_masculino_percentual(%)'] = 0.0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo_masculino_percentual(%)'][i] = eleitorado_facultativo_masculino_estados_1turno_percentual()

### Eleitorado facultativo feminino

In [ ]:
def eleitorado_facultativo_feminino_estados_1turno():
  return df_estados_1turno['eleitorado_facultativo'][i] - df_estados_1turno['eleitorado_facultativo_masculino'][i]

In [ ]:
df_estados_1turno['eleitorado_facultativo_feminino'] = 0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo_feminino'][i] = eleitorado_facultativo_feminino_estados_1turno()

In [ ]:
def eleitorado_facultativo_feminino_estados_1turno_percentual():
  return round(((df_estados_1turno['eleitorado_facultativo_feminino'][i] / df_estados_1turno['aptos'][i]) * 100), 2)

In [ ]:
df_estados_1turno['eleitorado_facultativo_feminino_percentual(%)'] = 0.0

for i in range(len(df_estados_1turno['SG_UF'])):
  df_estados_1turno['eleitorado_facultativo_feminino_percentual(%)'][i] = eleitorado_facultativo_feminino_estados_1turno_percentual()

## Estado civil do eleitorado

In [ ]:
estado_civil = df_turno_1_2020.groupby(['SG_UF', 'DS_ESTADO_CIVIL'])['QT_APTOS'].sum().reset_index()
df_pivot = estado_civil.pivot_table(index=['SG_UF'], columns='DS_ESTADO_CIVIL', values='QT_APTOS', fill_value=0).reset_index()
df_pivot = df_pivot.rename(columns={
       'SOLTEIRO':'solteiro', 'CASADO':'casado', 'DIVORCIADO':'divorciado',
       'VIÚVO':'viuvo', 'SEPARADO JUDICIALMENTE':'separado_judicialmente'})
df_pivot = df_pivot[['SG_UF', 'solteiro', 'casado',
      'divorciado', 'viuvo', 'separado_judicialmente']]
df_estados_1turno = df_estados_1turno.merge(df_pivot, how='left', left_on=['SG_UF'], right_on=['SG_UF'])
del estado_civil, df_pivot

### Estado civil percentual do eleitorado

In [ ]:
estados_civil = {
    'solteiro_percentual(%)': 'solteiro',
    'casado_percentual(%)': 'casado',
    'divorciado_percentual(%)': 'divorciado',
    'viuvo_percentual(%)': 'viuvo',
    'separado_judicialmente_percentual(%)': 'separado_judicialmente'
}

for estado_percentual, ec in estados_civil.items():
  df_estados_1turno[coluna_percentual] = (df_estados_1turno[ec] / df_estados_1turno['aptos'] * 100).round(2)

### Estado civil do eleitorado masculino

In [ ]:
# Filtra os dados apenas para o gênero masculino
df_turno_1_2020_masculino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'MASCULINO']

# Realiza a agregação dos dados por município e grau de escolaridade
agregado_masculino = df_turno_1_2020_masculino.groupby(['SG_UF', 'DS_ESTADO_CIVIL'])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = agregado_masculino.pivot_table(index=['SG_UF'], columns='DS_ESTADO_CIVIL', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.rename(columns={
       'SOLTEIRO':'solteiro_masculino', 'CASADO':'casado_masculino', 'DIVORCIADO':'divorciado_masculino',
       'VIÚVO':'viuvo_masculino', 'SEPARADO JUDICIALMENTE':'separado_judicialmente_masculino'})
pivot_masculino = pivot_masculino[['SG_UF', 'solteiro_masculino', 'casado_masculino',
      'divorciado_masculino', 'viuvo_masculino', 'separado_judicialmente_masculino']]

pivot_masculino = pivot_masculino.fillna(0)

for coluna in pivot_masculino:
  if (coluna != 'SG_UF'):
    pivot_masculino[coluna] = pivot_masculino[coluna].astype(int)
# Junta os dados no dataframe df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    'solteiro_masculino_percentual(%)': ['solteiro_masculino', 'solteiro'],
    'casado_masculino_percentual(%)': ['casado_masculino', 'casado'],
    'divorciado_masculino_percentual(%)': ['divorciado_masculino', 'divorciado'],
    'viuvo_masculino_percentual(%)': ['viuvo_masculino', 'viuvo'],
    'separado_judicialmente_masculino_percentual(%)': ['separado_judicialmente_masculino', 'separado_judicialmente']
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

### Estado civil do eleitorado feminino

In [ ]:
# Filtra os dados apenas para o gênero masculino
df_turno_1_2020_feminino = df_turno_1_2020[df_turno_1_2020['DS_GENERO'] == 'FEMININO']

# Realiza a agregação dos dados por município e grau de escolaridade
agregado_feminino = df_turno_1_2020_feminino.groupby(['SG_UF', 'DS_ESTADO_CIVIL'])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = agregado_feminino.pivot_table(index=['SG_UF'], columns='DS_ESTADO_CIVIL', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.rename(columns={
       'SOLTEIRO':'solteiro_feminino', 'CASADO':'casado_feminino', 'DIVORCIADO':'divorciado_feminino',
       'VIÚVO':'viuvo_feminino', 'SEPARADO JUDICIALMENTE':'separado_judicialmente_feminino'})
pivot_feminino = pivot_feminino[['SG_UF', 'solteiro_feminino', 'casado_feminino',
      'divorciado_feminino', 'viuvo_feminino', 'separado_judicialmente_feminino']]
# Junta os dados no dataframe df_estados_1turno
df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    'solteiro_feminino_percentual(%)': ['solteiro_feminino', 'solteiro'],
    'casado_feminino_percentual(%)': ['casado_feminino', 'casado'],
    'divorciado_feminino_percentual(%)': ['divorciado_feminino', 'divorciado'],
    'viuvo_feminino_percentual(%)': ['viuvo_feminino', 'viuvo'],
    'separado_judicialmente_feminino_percentual(%)': ['separado_judicialmente_feminino', 'separado_judicialmente']
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

## Eleitorado masculino solteiro por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_masculino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'MASCULINO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'SOLTEIRO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = df_turno_1_2020_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.fillna(0)
pivot_masculino = pivot_masculino.rename(columns={
       '16 anos':'16_anos_solteiro_masculino', '17 anos':'17_anos_solteiro_masculino', '18 anos':'18_anos_solteiro_masculino',
       '19 anos':'19_anos_solteiro_masculino', '20 anos':'20_anos_solteiro_masculino', '21 a 24 anos':'21_24_anos_solteiro_masculino',
       '25 a 29 anos':'25_29_anos_solteiro_masculino', '30 a 34 anos':'30_34_anos_solteiro_masculino', '35 a 39 anos':'35_39_anos_solteiro_masculino',
       '40 a 44 anos':'40_44_anos_solteiro_masculino', '45 a 49 anos':'45_49_anos_solteiro_masculino',
       '50 a 54 anos':'50_54_anos_solteiro_masculino', '55 a 59 anos':'55_59_anos_solteiro_masculino',
       '60 a 64 anos':'60_64_anos_solteiro_masculino', '65 a 69 anos':'65_69_anos_solteiro_masculino',
       '70 a 74 anos':'70_74_anos_solteiro_masculino', '75 a 79 anos':'75_79_anos_solteiro_masculino',
       '80 a 84 anos':'80_84_anos_solteiro_masculino', '85 a 89 anos':'85_89_anos_solteiro_masculino',
       '90 a 94 anos':'90_94_anos_solteiro_masculino', '95 a 99 anos':'95_99_anos_solteiro_masculino',
       '100 anos ou mais':'100_anos_solteiro_masculino',})

pivot_masculino = pivot_masculino[['SG_UF',
       '16_anos_solteiro_masculino', '17_anos_solteiro_masculino',
       '18_anos_solteiro_masculino', '19_anos_solteiro_masculino',
       '20_anos_solteiro_masculino', '21_24_anos_solteiro_masculino',
       '25_29_anos_solteiro_masculino', '30_34_anos_solteiro_masculino',
       '35_39_anos_solteiro_masculino', '40_44_anos_solteiro_masculino',
       '45_49_anos_solteiro_masculino', '50_54_anos_solteiro_masculino',
       '55_59_anos_solteiro_masculino', '60_64_anos_solteiro_masculino',
       '65_69_anos_solteiro_masculino', '70_74_anos_solteiro_masculino',
       '75_79_anos_solteiro_masculino', '80_84_anos_solteiro_masculino',
       '85_89_anos_solteiro_masculino', '90_94_anos_solteiro_masculino',
       '95_99_anos_solteiro_masculino', '100_anos_solteiro_masculino',]]

for i in pivot_masculino.columns:
  if ((i != 'SG_UF')):
    pivot_masculino[i] = pivot_masculino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    '16_anos_solteiro_masculino_percentual(%)':['16_anos_solteiro_masculino','16_anos'],
    '17_anos_solteiro_masculino_percentual(%)':['17_anos_solteiro_masculino','17_anos'],
    '18_anos_solteiro_masculino_percentual(%)':['18_anos_solteiro_masculino','18_anos'],
    '19_anos_solteiro_masculino_percentual(%)':['19_anos_solteiro_masculino','19_anos'],
    '20_anos_solteiro_masculino_percentual(%)':['20_anos_solteiro_masculino','20_anos'],
    '21_24_anos_solteiro_masculino_percentual(%)':['21_24_anos_solteiro_masculino','21_24_anos'],
    '25_29_anos_solteiro_masculino_percentual(%)':['25_29_anos_solteiro_masculino','25_29_anos'],
    '30_34_anos_solteiro_masculino_percentual(%)':['30_34_anos_solteiro_masculino','30_34_anos'],
    '35_39_anos_solteiro_masculino_percentual(%)':['35_39_anos_solteiro_masculino','35_39_anos'],
    '40_44_anos_solteiro_masculino_percentual(%)':['40_44_anos_solteiro_masculino','40_44_anos'],
    '45_49_anos_solteiro_masculino_percentual(%)':['45_49_anos_solteiro_masculino','45_49_anos'],
    '50_54_anos_solteiro_masculino_percentual(%)':['50_54_anos_solteiro_masculino','50_54_anos'],
    '55_59_anos_solteiro_masculino_percentual(%)':['55_59_anos_solteiro_masculino','55_59_anos'],
    '60_64_anos_solteiro_masculino_percentual(%)':['60_64_anos_solteiro_masculino','60_64_anos'],
    '65_69_anos_solteiro_masculino_percentual(%)':['65_69_anos_solteiro_masculino','65_69_anos'],
    '70_74_anos_solteiro_masculino_percentual(%)':['70_74_anos_solteiro_masculino','70_74_anos'],
    '75_79_anos_solteiro_masculino_percentual(%)':['75_79_anos_solteiro_masculino','75_79_anos'],
    '80_84_anos_solteiro_masculino_percentual(%)':['80_84_anos_solteiro_masculino','80_84_anos'],
    '85_89_anos_solteiro_masculino_percentual(%)':['85_89_anos_solteiro_masculino','85_89_anos'],
    '90_94_anos_solteiro_masculino_percentual(%)':['90_94_anos_solteiro_masculino','90_94_anos'],
    '95_99_anos_solteiro_masculino_percentual(%)':['95_99_anos_solteiro_masculino','95_99_anos'],
    '100_anos_solteiro_masculino_percentual(%)':['100_anos_solteiro_masculino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_masculino

## Eleitorado masculino casado por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e casado
df_turno_1_2020_masculino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'MASCULINO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'CASADO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = df_turno_1_2020_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.fillna(0)
pivot_masculino = pivot_masculino.rename(columns={
       '16 anos':'16_anos_casado_masculino', '17 anos':'17_anos_casado_masculino', '18 anos':'18_anos_casado_masculino',
       '19 anos':'19_anos_casado_masculino', '20 anos':'20_anos_casado_masculino', '21 a 24 anos':'21_24_anos_casado_masculino',
       '25 a 29 anos':'25_29_anos_casado_masculino', '30 a 34 anos':'30_34_anos_casado_masculino', '35 a 39 anos':'35_39_anos_casado_masculino',
       '40 a 44 anos':'40_44_anos_casado_masculino', '45 a 49 anos':'45_49_anos_casado_masculino',
       '50 a 54 anos':'50_54_anos_casado_masculino', '55 a 59 anos':'55_59_anos_casado_masculino',
       '60 a 64 anos':'60_64_anos_casado_masculino', '65 a 69 anos':'65_69_anos_casado_masculino',
       '70 a 74 anos':'70_74_anos_casado_masculino', '75 a 79 anos':'75_79_anos_casado_masculino',
       '80 a 84 anos':'80_84_anos_casado_masculino', '85 a 89 anos':'85_89_anos_casado_masculino',
       '90 a 94 anos':'90_94_anos_casado_masculino', '95 a 99 anos':'95_99_anos_casado_masculino',
       '100 anos ou mais':'100_anos_casado_masculino',})

pivot_masculino = pivot_masculino[['SG_UF',
       '16_anos_casado_masculino', '17_anos_casado_masculino',
       '18_anos_casado_masculino', '19_anos_casado_masculino',
       '20_anos_casado_masculino', '21_24_anos_casado_masculino',
       '25_29_anos_casado_masculino', '30_34_anos_casado_masculino',
       '35_39_anos_casado_masculino', '40_44_anos_casado_masculino',
       '45_49_anos_casado_masculino', '50_54_anos_casado_masculino',
       '55_59_anos_casado_masculino', '60_64_anos_casado_masculino',
       '65_69_anos_casado_masculino', '70_74_anos_casado_masculino',
       '75_79_anos_casado_masculino', '80_84_anos_casado_masculino',
       '85_89_anos_casado_masculino', '90_94_anos_casado_masculino',
       '95_99_anos_casado_masculino', '100_anos_casado_masculino',]]

for i in pivot_masculino.columns:
  if ((i != 'SG_UF')):
    pivot_masculino[i] = pivot_masculino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    '16_anos_casado_masculino_percentual(%)':['16_anos_casado_masculino','16_anos'],
    '17_anos_casado_masculino_percentual(%)':['17_anos_casado_masculino','17_anos'],
    '18_anos_casado_masculino_percentual(%)':['18_anos_casado_masculino','18_anos'],
    '19_anos_casado_masculino_percentual(%)':['19_anos_casado_masculino','19_anos'],
    '20_anos_casado_masculino_percentual(%)':['20_anos_casado_masculino','20_anos'],
    '21_24_anos_casado_masculino_percentual(%)':['21_24_anos_casado_masculino','21_24_anos'],
    '25_29_anos_casado_masculino_percentual(%)':['25_29_anos_casado_masculino','25_29_anos'],
    '30_34_anos_casado_masculino_percentual(%)':['30_34_anos_casado_masculino','30_34_anos'],
    '35_39_anos_casado_masculino_percentual(%)':['35_39_anos_casado_masculino','35_39_anos'],
    '40_44_anos_casado_masculino_percentual(%)':['40_44_anos_casado_masculino','40_44_anos'],
    '45_49_anos_casado_masculino_percentual(%)':['45_49_anos_casado_masculino','45_49_anos'],
    '50_54_anos_casado_masculino_percentual(%)':['50_54_anos_casado_masculino','50_54_anos'],
    '55_59_anos_casado_masculino_percentual(%)':['55_59_anos_casado_masculino','55_59_anos'],
    '60_64_anos_casado_masculino_percentual(%)':['60_64_anos_casado_masculino','60_64_anos'],
    '65_69_anos_casado_masculino_percentual(%)':['65_69_anos_casado_masculino','65_69_anos'],
    '70_74_anos_casado_masculino_percentual(%)':['70_74_anos_casado_masculino','70_74_anos'],
    '75_79_anos_casado_masculino_percentual(%)':['75_79_anos_casado_masculino','75_79_anos'],
    '80_84_anos_casado_masculino_percentual(%)':['80_84_anos_casado_masculino','80_84_anos'],
    '85_89_anos_casado_masculino_percentual(%)':['85_89_anos_casado_masculino','85_89_anos'],
    '90_94_anos_casado_masculino_percentual(%)':['90_94_anos_casado_masculino','90_94_anos'],
    '95_99_anos_casado_masculino_percentual(%)':['95_99_anos_casado_masculino','95_99_anos'],
    '100_anos_casado_masculino_percentual(%)':['100_anos_casado_masculino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

## Eleitorado masculino divorciado por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e casado
df_turno_1_2020_masculino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'MASCULINO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'DIVORCIADO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = df_turno_1_2020_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.fillna(0)
pivot_masculino = pivot_masculino.rename(columns={
       '16 anos':'16_anos_divorciado_masculino', '17 anos':'17_anos_divorciado_masculino', '18 anos':'18_anos_divorciado_masculino',
       '19 anos':'19_anos_divorciado_masculino', '20 anos':'20_anos_divorciado_masculino', '21 a 24 anos':'21_24_anos_divorciado_masculino',
       '25 a 29 anos':'25_29_anos_divorciado_masculino', '30 a 34 anos':'30_34_anos_divorciado_masculino', '35 a 39 anos':'35_39_anos_divorciado_masculino',
       '40 a 44 anos':'40_44_anos_divorciado_masculino', '45 a 49 anos':'45_49_anos_divorciado_masculino',
       '50 a 54 anos':'50_54_anos_divorciado_masculino', '55 a 59 anos':'55_59_anos_divorciado_masculino',
       '60 a 64 anos':'60_64_anos_divorciado_masculino', '65 a 69 anos':'65_69_anos_divorciado_masculino',
       '70 a 74 anos':'70_74_anos_divorciado_masculino', '75 a 79 anos':'75_79_anos_divorciado_masculino',
       '80 a 84 anos':'80_84_anos_divorciado_masculino', '85 a 89 anos':'85_89_anos_divorciado_masculino',
       '90 a 94 anos':'90_94_anos_divorciado_masculino', '95 a 99 anos':'95_99_anos_divorciado_masculino',
       '100 anos ou mais':'100_anos_divorciado_masculino',})

pivot_masculino = pivot_masculino[['SG_UF',
       '16_anos_divorciado_masculino', '17_anos_divorciado_masculino',
       '18_anos_divorciado_masculino', '19_anos_divorciado_masculino',
       '20_anos_divorciado_masculino', '21_24_anos_divorciado_masculino',
       '25_29_anos_divorciado_masculino', '30_34_anos_divorciado_masculino',
       '35_39_anos_divorciado_masculino', '40_44_anos_divorciado_masculino',
       '45_49_anos_divorciado_masculino', '50_54_anos_divorciado_masculino',
       '55_59_anos_divorciado_masculino', '60_64_anos_divorciado_masculino',
       '65_69_anos_divorciado_masculino', '70_74_anos_divorciado_masculino',
       '75_79_anos_divorciado_masculino', '80_84_anos_divorciado_masculino',
       '85_89_anos_divorciado_masculino', '90_94_anos_divorciado_masculino',
       '95_99_anos_divorciado_masculino', '100_anos_divorciado_masculino',]]

for i in pivot_masculino.columns:
  if ((i != 'SG_UF')):
    pivot_masculino[i] = pivot_masculino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    '16_anos_divorciado_masculino_percentual(%)':['16_anos_divorciado_masculino','16_anos'],
    '17_anos_divorciado_masculino_percentual(%)':['17_anos_divorciado_masculino','17_anos'],
    '18_anos_divorciado_masculino_percentual(%)':['18_anos_divorciado_masculino','18_anos'],
    '19_anos_divorciado_masculino_percentual(%)':['19_anos_divorciado_masculino','19_anos'],
    '20_anos_divorciado_masculino_percentual(%)':['20_anos_divorciado_masculino','20_anos'],
    '21_24_anos_divorciado_masculino_percentual(%)':['21_24_anos_divorciado_masculino','21_24_anos'],
    '25_29_anos_divorciado_masculino_percentual(%)':['25_29_anos_divorciado_masculino','25_29_anos'],
    '30_34_anos_divorciado_masculino_percentual(%)':['30_34_anos_divorciado_masculino','30_34_anos'],
    '35_39_anos_divorciado_masculino_percentual(%)':['35_39_anos_divorciado_masculino','35_39_anos'],
    '40_44_anos_divorciado_masculino_percentual(%)':['40_44_anos_divorciado_masculino','40_44_anos'],
    '45_49_anos_divorciado_masculino_percentual(%)':['45_49_anos_divorciado_masculino','45_49_anos'],
    '50_54_anos_divorciado_masculino_percentual(%)':['50_54_anos_divorciado_masculino','50_54_anos'],
    '55_59_anos_divorciado_masculino_percentual(%)':['55_59_anos_divorciado_masculino','55_59_anos'],
    '60_64_anos_divorciado_masculino_percentual(%)':['60_64_anos_divorciado_masculino','60_64_anos'],
    '65_69_anos_divorciado_masculino_percentual(%)':['65_69_anos_divorciado_masculino','65_69_anos'],
    '70_74_anos_divorciado_masculino_percentual(%)':['70_74_anos_divorciado_masculino','70_74_anos'],
    '75_79_anos_divorciado_masculino_percentual(%)':['75_79_anos_divorciado_masculino','75_79_anos'],
    '80_84_anos_divorciado_masculino_percentual(%)':['80_84_anos_divorciado_masculino','80_84_anos'],
    '85_89_anos_divorciado_masculino_percentual(%)':['85_89_anos_divorciado_masculino','85_89_anos'],
    '90_94_anos_divorciado_masculino_percentual(%)':['90_94_anos_divorciado_masculino','90_94_anos'],
    '95_99_anos_divorciado_masculino_percentual(%)':['95_99_anos_divorciado_masculino','95_99_anos'],
    '100_anos_divorciado_masculino_percentual(%)':['100_anos_divorciado_masculino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

## Eleitorado masculino viúvo por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e casado
df_turno_1_2020_masculino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'MASCULINO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'VIÚVO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = df_turno_1_2020_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.fillna(0)
pivot_masculino = pivot_masculino.rename(columns={
       '16 anos':'16_anos_viuvo_masculino', '17 anos':'17_anos_viuvo_masculino', '18 anos':'18_anos_viuvo_masculino',
       '19 anos':'19_anos_viuvo_masculino', '20 anos':'20_anos_viuvo_masculino', '21 a 24 anos':'21_24_anos_viuvo_masculino',
       '25 a 29 anos':'25_29_anos_viuvo_masculino', '30 a 34 anos':'30_34_anos_viuvo_masculino', '35 a 39 anos':'35_39_anos_viuvo_masculino',
       '40 a 44 anos':'40_44_anos_viuvo_masculino', '45 a 49 anos':'45_49_anos_viuvo_masculino',
       '50 a 54 anos':'50_54_anos_viuvo_masculino', '55 a 59 anos':'55_59_anos_viuvo_masculino',
       '60 a 64 anos':'60_64_anos_viuvo_masculino', '65 a 69 anos':'65_69_anos_viuvo_masculino',
       '70 a 74 anos':'70_74_anos_viuvo_masculino', '75 a 79 anos':'75_79_anos_viuvo_masculino',
       '80 a 84 anos':'80_84_anos_viuvo_masculino', '85 a 89 anos':'85_89_anos_viuvo_masculino',
       '90 a 94 anos':'90_94_anos_viuvo_masculino', '95 a 99 anos':'95_99_anos_viuvo_masculino',
       '100 anos ou mais':'100_anos_viuvo_masculino',})

pivot_masculino = pivot_masculino[['SG_UF',
       '16_anos_viuvo_masculino', '17_anos_viuvo_masculino',
       '18_anos_viuvo_masculino', '19_anos_viuvo_masculino',
       '20_anos_viuvo_masculino', '21_24_anos_viuvo_masculino',
       '25_29_anos_viuvo_masculino', '30_34_anos_viuvo_masculino',
       '35_39_anos_viuvo_masculino', '40_44_anos_viuvo_masculino',
       '45_49_anos_viuvo_masculino', '50_54_anos_viuvo_masculino',
       '55_59_anos_viuvo_masculino', '60_64_anos_viuvo_masculino',
       '65_69_anos_viuvo_masculino', '70_74_anos_viuvo_masculino',
       '75_79_anos_viuvo_masculino', '80_84_anos_viuvo_masculino',
       '85_89_anos_viuvo_masculino', '90_94_anos_viuvo_masculino',
       '95_99_anos_viuvo_masculino', '100_anos_viuvo_masculino',]]

for i in pivot_masculino.columns:
  if ((i != 'SG_UF')):
    pivot_masculino[i] = pivot_masculino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    '16_anos_viuvo_masculino_percentual(%)':['16_anos_viuvo_masculino','16_anos'],
    '17_anos_viuvo_masculino_percentual(%)':['17_anos_viuvo_masculino','17_anos'],
    '18_anos_viuvo_masculino_percentual(%)':['18_anos_viuvo_masculino','18_anos'],
    '19_anos_viuvo_masculino_percentual(%)':['19_anos_viuvo_masculino','19_anos'],
    '20_anos_viuvo_masculino_percentual(%)':['20_anos_viuvo_masculino','20_anos'],
    '21_24_anos_viuvo_masculino_percentual(%)':['21_24_anos_viuvo_masculino','21_24_anos'],
    '25_29_anos_viuvo_masculino_percentual(%)':['25_29_anos_viuvo_masculino','25_29_anos'],
    '30_34_anos_viuvo_masculino_percentual(%)':['30_34_anos_viuvo_masculino','30_34_anos'],
    '35_39_anos_viuvo_masculino_percentual(%)':['35_39_anos_viuvo_masculino','35_39_anos'],
    '40_44_anos_viuvo_masculino_percentual(%)':['40_44_anos_viuvo_masculino','40_44_anos'],
    '45_49_anos_viuvo_masculino_percentual(%)':['45_49_anos_viuvo_masculino','45_49_anos'],
    '50_54_anos_viuvo_masculino_percentual(%)':['50_54_anos_viuvo_masculino','50_54_anos'],
    '55_59_anos_viuvo_masculino_percentual(%)':['55_59_anos_viuvo_masculino','55_59_anos'],
    '60_64_anos_viuvo_masculino_percentual(%)':['60_64_anos_viuvo_masculino','60_64_anos'],
    '65_69_anos_viuvo_masculino_percentual(%)':['65_69_anos_viuvo_masculino','65_69_anos'],
    '70_74_anos_viuvo_masculino_percentual(%)':['70_74_anos_viuvo_masculino','70_74_anos'],
    '75_79_anos_viuvo_masculino_percentual(%)':['75_79_anos_viuvo_masculino','75_79_anos'],
    '80_84_anos_viuvo_masculino_percentual(%)':['80_84_anos_viuvo_masculino','80_84_anos'],
    '85_89_anos_viuvo_masculino_percentual(%)':['85_89_anos_viuvo_masculino','85_89_anos'],
    '90_94_anos_viuvo_masculino_percentual(%)':['90_94_anos_viuvo_masculino','90_94_anos'],
    '95_99_anos_viuvo_masculino_percentual(%)':['95_99_anos_viuvo_masculino','95_99_anos'],
    '100_anos_viuvo_masculino_percentual(%)':['100_anos_viuvo_masculino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

## Eleitorado masculino separado judicialmente por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e casado
df_turno_1_2020_masculino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'MASCULINO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'SEPARADO JUDICIALMENTE')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_masculino = df_turno_1_2020_masculino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_masculino = pivot_masculino.fillna(0)
pivot_masculino = pivot_masculino.rename(columns={
       '16 anos':'16_anos_separado_judicialmente_masculino', '17 anos':'17_anos_separado_judicialmente_masculino', '18 anos':'18_anos_separado_judicialmente_masculino',
       '19 anos':'19_anos_separado_judicialmente_masculino', '20 anos':'20_anos_separado_judicialmente_masculino', '21 a 24 anos':'21_24_anos_separado_judicialmente_masculino',
       '25 a 29 anos':'25_29_anos_separado_judicialmente_masculino', '30 a 34 anos':'30_34_anos_separado_judicialmente_masculino', '35 a 39 anos':'35_39_anos_separado_judicialmente_masculino',
       '40 a 44 anos':'40_44_anos_separado_judicialmente_masculino', '45 a 49 anos':'45_49_anos_separado_judicialmente_masculino',
       '50 a 54 anos':'50_54_anos_separado_judicialmente_masculino', '55 a 59 anos':'55_59_anos_separado_judicialmente_masculino',
       '60 a 64 anos':'60_64_anos_separado_judicialmente_masculino', '65 a 69 anos':'65_69_anos_separado_judicialmente_masculino',
       '70 a 74 anos':'70_74_anos_separado_judicialmente_masculino', '75 a 79 anos':'75_79_anos_separado_judicialmente_masculino',
       '80 a 84 anos':'80_84_anos_separado_judicialmente_masculino', '85 a 89 anos':'85_89_anos_separado_judicialmente_masculino',
       '90 a 94 anos':'90_94_anos_separado_judicialmente_masculino', '95 a 99 anos':'95_99_anos_separado_judicialmente_masculino',
       '100 anos ou mais':'100_anos_separado_judicialmente_masculino',})

pivot_masculino = pivot_masculino[['SG_UF',
       '16_anos_separado_judicialmente_masculino', '17_anos_separado_judicialmente_masculino',
       '18_anos_separado_judicialmente_masculino', '19_anos_separado_judicialmente_masculino',
       '20_anos_separado_judicialmente_masculino', '21_24_anos_separado_judicialmente_masculino',
       '25_29_anos_separado_judicialmente_masculino', '30_34_anos_separado_judicialmente_masculino',
       '35_39_anos_separado_judicialmente_masculino', '40_44_anos_separado_judicialmente_masculino',
       '45_49_anos_separado_judicialmente_masculino', '50_54_anos_separado_judicialmente_masculino',
       '55_59_anos_separado_judicialmente_masculino', '60_64_anos_separado_judicialmente_masculino',
       '65_69_anos_separado_judicialmente_masculino', '70_74_anos_separado_judicialmente_masculino',
       '75_79_anos_separado_judicialmente_masculino', '80_84_anos_separado_judicialmente_masculino',
       '85_89_anos_separado_judicialmente_masculino', '90_94_anos_separado_judicialmente_masculino',
       '95_99_anos_separado_judicialmente_masculino', '100_anos_separado_judicialmente_masculino',]]

for i in pivot_masculino.columns:
  if ((i != 'SG_UF')):
    pivot_masculino[i] = pivot_masculino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_masculino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_masculino = {
    '16_anos_separado_judicialmente_masculino_percentual(%)':['16_anos_separado_judicialmente_masculino','16_anos'],
    '17_anos_separado_judicialmente_masculino_percentual(%)':['17_anos_separado_judicialmente_masculino','17_anos'],
    '18_anos_separado_judicialmente_masculino_percentual(%)':['18_anos_separado_judicialmente_masculino','18_anos'],
    '19_anos_separado_judicialmente_masculino_percentual(%)':['19_anos_separado_judicialmente_masculino','19_anos'],
    '20_anos_separado_judicialmente_masculino_percentual(%)':['20_anos_separado_judicialmente_masculino','20_anos'],
    '21_24_anos_separado_judicialmente_masculino_percentual(%)':['21_24_anos_separado_judicialmente_masculino','21_24_anos'],
    '25_29_anos_separado_judicialmente_masculino_percentual(%)':['25_29_anos_separado_judicialmente_masculino','25_29_anos'],
    '30_34_anos_separado_judicialmente_masculino_percentual(%)':['30_34_anos_separado_judicialmente_masculino','30_34_anos'],
    '35_39_anos_separado_judicialmente_masculino_percentual(%)':['35_39_anos_separado_judicialmente_masculino','35_39_anos'],
    '40_44_anos_separado_judicialmente_masculino_percentual(%)':['40_44_anos_separado_judicialmente_masculino','40_44_anos'],
    '45_49_anos_separado_judicialmente_masculino_percentual(%)':['45_49_anos_separado_judicialmente_masculino','45_49_anos'],
    '50_54_anos_separado_judicialmente_masculino_percentual(%)':['50_54_anos_separado_judicialmente_masculino','50_54_anos'],
    '55_59_anos_separado_judicialmente_masculino_percentual(%)':['55_59_anos_separado_judicialmente_masculino','55_59_anos'],
    '60_64_anos_separado_judicialmente_masculino_percentual(%)':['60_64_anos_separado_judicialmente_masculino','60_64_anos'],
    '65_69_anos_separado_judicialmente_masculino_percentual(%)':['65_69_anos_separado_judicialmente_masculino','65_69_anos'],
    '70_74_anos_separado_judicialmente_masculino_percentual(%)':['70_74_anos_separado_judicialmente_masculino','70_74_anos'],
    '75_79_anos_separado_judicialmente_masculino_percentual(%)':['75_79_anos_separado_judicialmente_masculino','75_79_anos'],
    '80_84_anos_separado_judicialmente_masculino_percentual(%)':['80_84_anos_separado_judicialmente_masculino','80_84_anos'],
    '85_89_anos_separado_judicialmente_masculino_percentual(%)':['85_89_anos_separado_judicialmente_masculino','85_89_anos'],
    '90_94_anos_separado_judicialmente_masculino_percentual(%)':['90_94_anos_separado_judicialmente_masculino','90_94_anos'],
    '95_99_anos_separado_judicialmente_masculino_percentual(%)':['95_99_anos_separado_judicialmente_masculino','95_99_anos'],
    '100_anos_separado_judicialmente_masculino_percentual(%)':['100_anos_separado_judicialmente_masculino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_masculino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

## Eleitorado feminino solteiro por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_feminino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'FEMININO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'SOLTEIRO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = df_turno_1_2020_feminino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.fillna(0)
pivot_feminino = pivot_feminino.rename(columns={
       '16 anos':'16_anos_solteiro_feminino', '17 anos':'17_anos_solteiro_feminino', '18 anos':'18_anos_solteiro_feminino',
       '19 anos':'19_anos_solteiro_feminino', '20 anos':'20_anos_solteiro_feminino', '21 a 24 anos':'21_24_anos_solteiro_feminino',
       '25 a 29 anos':'25_29_anos_solteiro_feminino', '30 a 34 anos':'30_34_anos_solteiro_feminino', '35 a 39 anos':'35_39_anos_solteiro_feminino',
       '40 a 44 anos':'40_44_anos_solteiro_feminino', '45 a 49 anos':'45_49_anos_solteiro_feminino',
       '50 a 54 anos':'50_54_anos_solteiro_feminino', '55 a 59 anos':'55_59_anos_solteiro_feminino',
       '60 a 64 anos':'60_64_anos_solteiro_feminino', '65 a 69 anos':'65_69_anos_solteiro_feminino',
       '70 a 74 anos':'70_74_anos_solteiro_feminino', '75 a 79 anos':'75_79_anos_solteiro_feminino',
       '80 a 84 anos':'80_84_anos_solteiro_feminino', '85 a 89 anos':'85_89_anos_solteiro_feminino',
       '90 a 94 anos':'90_94_anos_solteiro_feminino', '95 a 99 anos':'95_99_anos_solteiro_feminino',
       '100 anos ou mais':'100_anos_solteiro_feminino',})

pivot_feminino = pivot_feminino[['SG_UF',
       '16_anos_solteiro_feminino', '17_anos_solteiro_feminino',
       '18_anos_solteiro_feminino', '19_anos_solteiro_feminino',
       '20_anos_solteiro_feminino', '21_24_anos_solteiro_feminino',
       '25_29_anos_solteiro_feminino', '30_34_anos_solteiro_feminino',
       '35_39_anos_solteiro_feminino', '40_44_anos_solteiro_feminino',
       '45_49_anos_solteiro_feminino', '50_54_anos_solteiro_feminino',
       '55_59_anos_solteiro_feminino', '60_64_anos_solteiro_feminino',
       '65_69_anos_solteiro_feminino', '70_74_anos_solteiro_feminino',
       '75_79_anos_solteiro_feminino', '80_84_anos_solteiro_feminino',
       '85_89_anos_solteiro_feminino', '90_94_anos_solteiro_feminino',
       '95_99_anos_solteiro_feminino', '100_anos_solteiro_feminino',]]

for i in pivot_feminino.columns:
  if ((i != 'SG_UF')):
    pivot_feminino[i] = pivot_feminino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    '16_anos_solteiro_feminino_percentual(%)':['16_anos_solteiro_feminino','16_anos'],
    '17_anos_solteiro_feminino_percentual(%)':['17_anos_solteiro_feminino','17_anos'],
    '18_anos_solteiro_feminino_percentual(%)':['18_anos_solteiro_feminino','18_anos'],
    '19_anos_solteiro_feminino_percentual(%)':['19_anos_solteiro_feminino','19_anos'],
    '20_anos_solteiro_feminino_percentual(%)':['20_anos_solteiro_feminino','20_anos'],
    '21_24_anos_solteiro_feminino_percentual(%)':['21_24_anos_solteiro_feminino','21_24_anos'],
    '25_29_anos_solteiro_feminino_percentual(%)':['25_29_anos_solteiro_feminino','25_29_anos'],
    '30_34_anos_solteiro_feminino_percentual(%)':['30_34_anos_solteiro_feminino','30_34_anos'],
    '35_39_anos_solteiro_feminino_percentual(%)':['35_39_anos_solteiro_feminino','35_39_anos'],
    '40_44_anos_solteiro_feminino_percentual(%)':['40_44_anos_solteiro_feminino','40_44_anos'],
    '45_49_anos_solteiro_feminino_percentual(%)':['45_49_anos_solteiro_feminino','45_49_anos'],
    '50_54_anos_solteiro_feminino_percentual(%)':['50_54_anos_solteiro_feminino','50_54_anos'],
    '55_59_anos_solteiro_feminino_percentual(%)':['55_59_anos_solteiro_feminino','55_59_anos'],
    '60_64_anos_solteiro_feminino_percentual(%)':['60_64_anos_solteiro_feminino','60_64_anos'],
    '65_69_anos_solteiro_feminino_percentual(%)':['65_69_anos_solteiro_feminino','65_69_anos'],
    '70_74_anos_solteiro_feminino_percentual(%)':['70_74_anos_solteiro_feminino','70_74_anos'],
    '75_79_anos_solteiro_feminino_percentual(%)':['75_79_anos_solteiro_feminino','75_79_anos'],
    '80_84_anos_solteiro_feminino_percentual(%)':['80_84_anos_solteiro_feminino','80_84_anos'],
    '85_89_anos_solteiro_feminino_percentual(%)':['85_89_anos_solteiro_feminino','85_89_anos'],
    '90_94_anos_solteiro_feminino_percentual(%)':['90_94_anos_solteiro_feminino','90_94_anos'],
    '95_99_anos_solteiro_feminino_percentual(%)':['95_99_anos_solteiro_feminino','95_99_anos'],
    '100_anos_solteiro_feminino_percentual(%)':['100_anos_solteiro_feminino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_feminino

## Eleitorado feminino casado por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_feminino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'FEMININO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'CASADO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = df_turno_1_2020_feminino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.fillna(0)
pivot_feminino = pivot_feminino.rename(columns={
       '16 anos':'16_anos_casado_feminino', '17 anos':'17_anos_casado_feminino', '18 anos':'18_anos_casado_feminino',
       '19 anos':'19_anos_casado_feminino', '20 anos':'20_anos_casado_feminino', '21 a 24 anos':'21_24_anos_casado_feminino',
       '25 a 29 anos':'25_29_anos_casado_feminino', '30 a 34 anos':'30_34_anos_casado_feminino', '35 a 39 anos':'35_39_anos_casado_feminino',
       '40 a 44 anos':'40_44_anos_casado_feminino', '45 a 49 anos':'45_49_anos_casado_feminino',
       '50 a 54 anos':'50_54_anos_casado_feminino', '55 a 59 anos':'55_59_anos_casado_feminino',
       '60 a 64 anos':'60_64_anos_casado_feminino', '65 a 69 anos':'65_69_anos_casado_feminino',
       '70 a 74 anos':'70_74_anos_casado_feminino', '75 a 79 anos':'75_79_anos_casado_feminino',
       '80 a 84 anos':'80_84_anos_casado_feminino', '85 a 89 anos':'85_89_anos_casado_feminino',
       '90 a 94 anos':'90_94_anos_casado_feminino', '95 a 99 anos':'95_99_anos_casado_feminino',
       '100 anos ou mais':'100_anos_casado_feminino',})

pivot_feminino = pivot_feminino[['SG_UF',
       '16_anos_casado_feminino', '17_anos_casado_feminino',
       '18_anos_casado_feminino', '19_anos_casado_feminino',
       '20_anos_casado_feminino', '21_24_anos_casado_feminino',
       '25_29_anos_casado_feminino', '30_34_anos_casado_feminino',
       '35_39_anos_casado_feminino', '40_44_anos_casado_feminino',
       '45_49_anos_casado_feminino', '50_54_anos_casado_feminino',
       '55_59_anos_casado_feminino', '60_64_anos_casado_feminino',
       '65_69_anos_casado_feminino', '70_74_anos_casado_feminino',
       '75_79_anos_casado_feminino', '80_84_anos_casado_feminino',
       '85_89_anos_casado_feminino', '90_94_anos_casado_feminino',
       '95_99_anos_casado_feminino', '100_anos_casado_feminino',]]

for i in pivot_feminino.columns:
  if ((i != 'SG_UF')):
    pivot_feminino[i] = pivot_feminino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    '16_anos_casado_feminino_percentual(%)':['16_anos_casado_feminino','16_anos'],
    '17_anos_casado_feminino_percentual(%)':['17_anos_casado_feminino','17_anos'],
    '18_anos_casado_feminino_percentual(%)':['18_anos_casado_feminino','18_anos'],
    '19_anos_casado_feminino_percentual(%)':['19_anos_casado_feminino','19_anos'],
    '20_anos_casado_feminino_percentual(%)':['20_anos_casado_feminino','20_anos'],
    '21_24_anos_casado_feminino_percentual(%)':['21_24_anos_casado_feminino','21_24_anos'],
    '25_29_anos_casado_feminino_percentual(%)':['25_29_anos_casado_feminino','25_29_anos'],
    '30_34_anos_casado_feminino_percentual(%)':['30_34_anos_casado_feminino','30_34_anos'],
    '35_39_anos_casado_feminino_percentual(%)':['35_39_anos_casado_feminino','35_39_anos'],
    '40_44_anos_casado_feminino_percentual(%)':['40_44_anos_casado_feminino','40_44_anos'],
    '45_49_anos_casado_feminino_percentual(%)':['45_49_anos_casado_feminino','45_49_anos'],
    '50_54_anos_casado_feminino_percentual(%)':['50_54_anos_casado_feminino','50_54_anos'],
    '55_59_anos_casado_feminino_percentual(%)':['55_59_anos_casado_feminino','55_59_anos'],
    '60_64_anos_casado_feminino_percentual(%)':['60_64_anos_casado_feminino','60_64_anos'],
    '65_69_anos_casado_feminino_percentual(%)':['65_69_anos_casado_feminino','65_69_anos'],
    '70_74_anos_casado_feminino_percentual(%)':['70_74_anos_casado_feminino','70_74_anos'],
    '75_79_anos_casado_feminino_percentual(%)':['75_79_anos_casado_feminino','75_79_anos'],
    '80_84_anos_casado_feminino_percentual(%)':['80_84_anos_casado_feminino','80_84_anos'],
    '85_89_anos_casado_feminino_percentual(%)':['85_89_anos_casado_feminino','85_89_anos'],
    '90_94_anos_casado_feminino_percentual(%)':['90_94_anos_casado_feminino','90_94_anos'],
    '95_99_anos_casado_feminino_percentual(%)':['95_99_anos_casado_feminino','95_99_anos'],
    '100_anos_casado_feminino_percentual(%)':['100_anos_casado_feminino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_feminino

## Eleitorado feminino divorciado por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_feminino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'FEMININO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'DIVORCIADO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = df_turno_1_2020_feminino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.fillna(0)
pivot_feminino = pivot_feminino.rename(columns={
       '16 anos':'16_anos_divorciado_feminino', '17 anos':'17_anos_divorciado_feminino', '18 anos':'18_anos_divorciado_feminino',
       '19 anos':'19_anos_divorciado_feminino', '20 anos':'20_anos_divorciado_feminino', '21 a 24 anos':'21_24_anos_divorciado_feminino',
       '25 a 29 anos':'25_29_anos_divorciado_feminino', '30 a 34 anos':'30_34_anos_divorciado_feminino', '35 a 39 anos':'35_39_anos_divorciado_feminino',
       '40 a 44 anos':'40_44_anos_divorciado_feminino', '45 a 49 anos':'45_49_anos_divorciado_feminino',
       '50 a 54 anos':'50_54_anos_divorciado_feminino', '55 a 59 anos':'55_59_anos_divorciado_feminino',
       '60 a 64 anos':'60_64_anos_divorciado_feminino', '65 a 69 anos':'65_69_anos_divorciado_feminino',
       '70 a 74 anos':'70_74_anos_divorciado_feminino', '75 a 79 anos':'75_79_anos_divorciado_feminino',
       '80 a 84 anos':'80_84_anos_divorciado_feminino', '85 a 89 anos':'85_89_anos_divorciado_feminino',
       '90 a 94 anos':'90_94_anos_divorciado_feminino', '95 a 99 anos':'95_99_anos_divorciado_feminino',
       '100 anos ou mais':'100_anos_divorciado_feminino',})

pivot_feminino = pivot_feminino[['SG_UF',
       '16_anos_divorciado_feminino', '17_anos_divorciado_feminino',
       '18_anos_divorciado_feminino', '19_anos_divorciado_feminino',
       '20_anos_divorciado_feminino', '21_24_anos_divorciado_feminino',
       '25_29_anos_divorciado_feminino', '30_34_anos_divorciado_feminino',
       '35_39_anos_divorciado_feminino', '40_44_anos_divorciado_feminino',
       '45_49_anos_divorciado_feminino', '50_54_anos_divorciado_feminino',
       '55_59_anos_divorciado_feminino', '60_64_anos_divorciado_feminino',
       '65_69_anos_divorciado_feminino', '70_74_anos_divorciado_feminino',
       '75_79_anos_divorciado_feminino', '80_84_anos_divorciado_feminino',
       '85_89_anos_divorciado_feminino', '90_94_anos_divorciado_feminino',
       '95_99_anos_divorciado_feminino', '100_anos_divorciado_feminino',]]

for i in pivot_feminino.columns:
  if ((i != 'SG_UF')):
    pivot_feminino[i] = pivot_feminino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    '16_anos_divorciado_feminino_percentual(%)':['16_anos_divorciado_feminino','16_anos'],
    '17_anos_divorciado_feminino_percentual(%)':['17_anos_divorciado_feminino','17_anos'],
    '18_anos_divorciado_feminino_percentual(%)':['18_anos_divorciado_feminino','18_anos'],
    '19_anos_divorciado_feminino_percentual(%)':['19_anos_divorciado_feminino','19_anos'],
    '20_anos_divorciado_feminino_percentual(%)':['20_anos_divorciado_feminino','20_anos'],
    '21_24_anos_divorciado_feminino_percentual(%)':['21_24_anos_divorciado_feminino','21_24_anos'],
    '25_29_anos_divorciado_feminino_percentual(%)':['25_29_anos_divorciado_feminino','25_29_anos'],
    '30_34_anos_divorciado_feminino_percentual(%)':['30_34_anos_divorciado_feminino','30_34_anos'],
    '35_39_anos_divorciado_feminino_percentual(%)':['35_39_anos_divorciado_feminino','35_39_anos'],
    '40_44_anos_divorciado_feminino_percentual(%)':['40_44_anos_divorciado_feminino','40_44_anos'],
    '45_49_anos_divorciado_feminino_percentual(%)':['45_49_anos_divorciado_feminino','45_49_anos'],
    '50_54_anos_divorciado_feminino_percentual(%)':['50_54_anos_divorciado_feminino','50_54_anos'],
    '55_59_anos_divorciado_feminino_percentual(%)':['55_59_anos_divorciado_feminino','55_59_anos'],
    '60_64_anos_divorciado_feminino_percentual(%)':['60_64_anos_divorciado_feminino','60_64_anos'],
    '65_69_anos_divorciado_feminino_percentual(%)':['65_69_anos_divorciado_feminino','65_69_anos'],
    '70_74_anos_divorciado_feminino_percentual(%)':['70_74_anos_divorciado_feminino','70_74_anos'],
    '75_79_anos_divorciado_feminino_percentual(%)':['75_79_anos_divorciado_feminino','75_79_anos'],
    '80_84_anos_divorciado_feminino_percentual(%)':['80_84_anos_divorciado_feminino','80_84_anos'],
    '85_89_anos_divorciado_feminino_percentual(%)':['85_89_anos_divorciado_feminino','85_89_anos'],
    '90_94_anos_divorciado_feminino_percentual(%)':['90_94_anos_divorciado_feminino','90_94_anos'],
    '95_99_anos_divorciado_feminino_percentual(%)':['95_99_anos_divorciado_feminino','95_99_anos'],
    '100_anos_divorciado_feminino_percentual(%)':['100_anos_divorciado_feminino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_feminino

## Eleitorado feminino viúvo por faixa etária

In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_feminino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'FEMININO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'VIÚVO')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = df_turno_1_2020_feminino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.fillna(0)
pivot_feminino = pivot_feminino.rename(columns={
       '16 anos':'16_anos_viuvo_feminino', '17 anos':'17_anos_viuvo_feminino', '18 anos':'18_anos_viuvo_feminino',
       '19 anos':'19_anos_viuvo_feminino', '20 anos':'20_anos_viuvo_feminino', '21 a 24 anos':'21_24_anos_viuvo_feminino',
       '25 a 29 anos':'25_29_anos_viuvo_feminino', '30 a 34 anos':'30_34_anos_viuvo_feminino', '35 a 39 anos':'35_39_anos_viuvo_feminino',
       '40 a 44 anos':'40_44_anos_viuvo_feminino', '45 a 49 anos':'45_49_anos_viuvo_feminino',
       '50 a 54 anos':'50_54_anos_viuvo_feminino', '55 a 59 anos':'55_59_anos_viuvo_feminino',
       '60 a 64 anos':'60_64_anos_viuvo_feminino', '65 a 69 anos':'65_69_anos_viuvo_feminino',
       '70 a 74 anos':'70_74_anos_viuvo_feminino', '75 a 79 anos':'75_79_anos_viuvo_feminino',
       '80 a 84 anos':'80_84_anos_viuvo_feminino', '85 a 89 anos':'85_89_anos_viuvo_feminino',
       '90 a 94 anos':'90_94_anos_viuvo_feminino', '95 a 99 anos':'95_99_anos_viuvo_feminino',
       '100 anos ou mais':'100_anos_viuvo_feminino',})

pivot_feminino = pivot_feminino[['SG_UF',
       '16_anos_viuvo_feminino', '17_anos_viuvo_feminino',
       '18_anos_viuvo_feminino', '19_anos_viuvo_feminino',
       '20_anos_viuvo_feminino', '21_24_anos_viuvo_feminino',
       '25_29_anos_viuvo_feminino', '30_34_anos_viuvo_feminino',
       '35_39_anos_viuvo_feminino', '40_44_anos_viuvo_feminino',
       '45_49_anos_viuvo_feminino', '50_54_anos_viuvo_feminino',
       '55_59_anos_viuvo_feminino', '60_64_anos_viuvo_feminino',
       '65_69_anos_viuvo_feminino', '70_74_anos_viuvo_feminino',
       '75_79_anos_viuvo_feminino', '80_84_anos_viuvo_feminino',
       '85_89_anos_viuvo_feminino', '90_94_anos_viuvo_feminino',
       '95_99_anos_viuvo_feminino', '100_anos_viuvo_feminino',]]

for i in pivot_feminino.columns:
  if ((i != 'SG_UF')):
    pivot_feminino[i] = pivot_feminino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    '16_anos_viuvo_feminino_percentual(%)':['16_anos_viuvo_feminino','16_anos'],
    '17_anos_viuvo_feminino_percentual(%)':['17_anos_viuvo_feminino','17_anos'],
    '18_anos_viuvo_feminino_percentual(%)':['18_anos_viuvo_feminino','18_anos'],
    '19_anos_viuvo_feminino_percentual(%)':['19_anos_viuvo_feminino','19_anos'],
    '20_anos_viuvo_feminino_percentual(%)':['20_anos_viuvo_feminino','20_anos'],
    '21_24_anos_viuvo_feminino_percentual(%)':['21_24_anos_viuvo_feminino','21_24_anos'],
    '25_29_anos_viuvo_feminino_percentual(%)':['25_29_anos_viuvo_feminino','25_29_anos'],
    '30_34_anos_viuvo_feminino_percentual(%)':['30_34_anos_viuvo_feminino','30_34_anos'],
    '35_39_anos_viuvo_feminino_percentual(%)':['35_39_anos_viuvo_feminino','35_39_anos'],
    '40_44_anos_viuvo_feminino_percentual(%)':['40_44_anos_viuvo_feminino','40_44_anos'],
    '45_49_anos_viuvo_feminino_percentual(%)':['45_49_anos_viuvo_feminino','45_49_anos'],
    '50_54_anos_viuvo_feminino_percentual(%)':['50_54_anos_viuvo_feminino','50_54_anos'],
    '55_59_anos_viuvo_feminino_percentual(%)':['55_59_anos_viuvo_feminino','55_59_anos'],
    '60_64_anos_viuvo_feminino_percentual(%)':['60_64_anos_viuvo_feminino','60_64_anos'],
    '65_69_anos_viuvo_feminino_percentual(%)':['65_69_anos_viuvo_feminino','65_69_anos'],
    '70_74_anos_viuvo_feminino_percentual(%)':['70_74_anos_viuvo_feminino','70_74_anos'],
    '75_79_anos_viuvo_feminino_percentual(%)':['75_79_anos_viuvo_feminino','75_79_anos'],
    '80_84_anos_viuvo_feminino_percentual(%)':['80_84_anos_viuvo_feminino','80_84_anos'],
    '85_89_anos_viuvo_feminino_percentual(%)':['85_89_anos_viuvo_feminino','85_89_anos'],
    '90_94_anos_viuvo_feminino_percentual(%)':['90_94_anos_viuvo_feminino','90_94_anos'],
    '95_99_anos_viuvo_feminino_percentual(%)':['95_99_anos_viuvo_feminino','95_99_anos'],
    '100_anos_viuvo_feminino_percentual(%)':['100_anos_viuvo_feminino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_feminino

## Eleitorado feminino separado judicialmente por faixa etária


In [ ]:
# Filtra os dados apenas para o gênero masculino e solteiro
df_turno_1_2020_feminino = df_turno_1_2020[(df_turno_1_2020['DS_GENERO'] == 'FEMININO') & (df_turno_1_2020['DS_ESTADO_CIVIL'] == 'SEPARADO JUDICIALMENTE')].groupby(['SG_UF', 'DS_FAIXA_ETARIA',])['QT_APTOS'].sum().reset_index()

# Realiza o pivot para transformar as colunas de grau de escolaridade em colunas do dataframe
pivot_feminino = df_turno_1_2020_feminino.pivot_table(index=['SG_UF'], columns='DS_FAIXA_ETARIA', values='QT_APTOS').reset_index()
pivot_feminino = pivot_feminino.fillna(0)
pivot_feminino = pivot_feminino.rename(columns={
       '16 anos':'16_anos_separado_judicialmente_feminino', '17 anos':'17_anos_separado_judicialmente_feminino', '18 anos':'18_anos_separado_judicialmente_feminino',
       '19 anos':'19_anos_separado_judicialmente_feminino', '20 anos':'20_anos_separado_judicialmente_feminino', '21 a 24 anos':'21_24_anos_separado_judicialmente_feminino',
       '25 a 29 anos':'25_29_anos_separado_judicialmente_feminino', '30 a 34 anos':'30_34_anos_separado_judicialmente_feminino', '35 a 39 anos':'35_39_anos_separado_judicialmente_feminino',
       '40 a 44 anos':'40_44_anos_separado_judicialmente_feminino', '45 a 49 anos':'45_49_anos_separado_judicialmente_feminino',
       '50 a 54 anos':'50_54_anos_separado_judicialmente_feminino', '55 a 59 anos':'55_59_anos_separado_judicialmente_feminino',
       '60 a 64 anos':'60_64_anos_separado_judicialmente_feminino', '65 a 69 anos':'65_69_anos_separado_judicialmente_feminino',
       '70 a 74 anos':'70_74_anos_separado_judicialmente_feminino', '75 a 79 anos':'75_79_anos_separado_judicialmente_feminino',
       '80 a 84 anos':'80_84_anos_separado_judicialmente_feminino', '85 a 89 anos':'85_89_anos_separado_judicialmente_feminino',
       '90 a 94 anos':'90_94_anos_separado_judicialmente_feminino', '95 a 99 anos':'95_99_anos_separado_judicialmente_feminino',
       '100 anos ou mais':'100_anos_separado_judicialmente_feminino',})

pivot_feminino = pivot_feminino[['SG_UF',
       '16_anos_separado_judicialmente_feminino', '17_anos_separado_judicialmente_feminino',
       '18_anos_separado_judicialmente_feminino', '19_anos_separado_judicialmente_feminino',
       '20_anos_separado_judicialmente_feminino', '21_24_anos_separado_judicialmente_feminino',
       '25_29_anos_separado_judicialmente_feminino', '30_34_anos_separado_judicialmente_feminino',
       '35_39_anos_separado_judicialmente_feminino', '40_44_anos_separado_judicialmente_feminino',
       '45_49_anos_separado_judicialmente_feminino', '50_54_anos_separado_judicialmente_feminino',
       '55_59_anos_separado_judicialmente_feminino', '60_64_anos_separado_judicialmente_feminino',
       '65_69_anos_separado_judicialmente_feminino', '70_74_anos_separado_judicialmente_feminino',
       '75_79_anos_separado_judicialmente_feminino', '80_84_anos_separado_judicialmente_feminino',
       '85_89_anos_separado_judicialmente_feminino', '90_94_anos_separado_judicialmente_feminino',
       '95_99_anos_separado_judicialmente_feminino', '100_anos_separado_judicialmente_feminino',]]

for i in pivot_feminino.columns:
  if ((i != 'SG_UF')):
    pivot_feminino[i] = pivot_feminino[i].astype(int)

df_estados_1turno = df_estados_1turno.merge(pivot_feminino, on=['SG_UF'], how='left')

In [ ]:
estados_civil_percentual_feminino = {
    '16_anos_separado_judicialmente_feminino_percentual(%)':['16_anos_separado_judicialmente_feminino','16_anos'],
    '17_anos_separado_judicialmente_feminino_percentual(%)':['17_anos_separado_judicialmente_feminino','17_anos'],
    '18_anos_separado_judicialmente_feminino_percentual(%)':['18_anos_separado_judicialmente_feminino','18_anos'],
    '19_anos_separado_judicialmente_feminino_percentual(%)':['19_anos_separado_judicialmente_feminino','19_anos'],
    '20_anos_separado_judicialmente_feminino_percentual(%)':['20_anos_separado_judicialmente_feminino','20_anos'],
    '21_24_anos_separado_judicialmente_feminino_percentual(%)':['21_24_anos_separado_judicialmente_feminino','21_24_anos'],
    '25_29_anos_separado_judicialmente_feminino_percentual(%)':['25_29_anos_separado_judicialmente_feminino','25_29_anos'],
    '30_34_anos_separado_judicialmente_feminino_percentual(%)':['30_34_anos_separado_judicialmente_feminino','30_34_anos'],
    '35_39_anos_separado_judicialmente_feminino_percentual(%)':['35_39_anos_separado_judicialmente_feminino','35_39_anos'],
    '40_44_anos_separado_judicialmente_feminino_percentual(%)':['40_44_anos_separado_judicialmente_feminino','40_44_anos'],
    '45_49_anos_separado_judicialmente_feminino_percentual(%)':['45_49_anos_separado_judicialmente_feminino','45_49_anos'],
    '50_54_anos_separado_judicialmente_feminino_percentual(%)':['50_54_anos_separado_judicialmente_feminino','50_54_anos'],
    '55_59_anos_separado_judicialmente_feminino_percentual(%)':['55_59_anos_separado_judicialmente_feminino','55_59_anos'],
    '60_64_anos_separado_judicialmente_feminino_percentual(%)':['60_64_anos_separado_judicialmente_feminino','60_64_anos'],
    '65_69_anos_separado_judicialmente_feminino_percentual(%)':['65_69_anos_separado_judicialmente_feminino','65_69_anos'],
    '70_74_anos_separado_judicialmente_feminino_percentual(%)':['70_74_anos_separado_judicialmente_feminino','70_74_anos'],
    '75_79_anos_separado_judicialmente_feminino_percentual(%)':['75_79_anos_separado_judicialmente_feminino','75_79_anos'],
    '80_84_anos_separado_judicialmente_feminino_percentual(%)':['80_84_anos_separado_judicialmente_feminino','80_84_anos'],
    '85_89_anos_separado_judicialmente_feminino_percentual(%)':['85_89_anos_separado_judicialmente_feminino','85_89_anos'],
    '90_94_anos_separado_judicialmente_feminino_percentual(%)':['90_94_anos_separado_judicialmente_feminino','90_94_anos'],
    '95_99_anos_separado_judicialmente_feminino_percentual(%)':['95_99_anos_separado_judicialmente_feminino','95_99_anos'],
    '100_anos_separado_judicialmente_feminino_percentual(%)':['100_anos_separado_judicialmente_feminino','100_anos'],
}

for estado_percentual, ec in estados_civil_percentual_feminino.items():
  df_estados_1turno[estado_percentual] = (df_estados_1turno[ec[0]] / df_estados_1turno[ec[1]] * 100).round(2)

del estados_civil_percentual_feminino

## Salvamento do dataframe final

In [ ]:
from google.colab import files
df_estados_1turno = df_estados_1turno.rename(columns={'SG_UF':'estado'})
df_estados_1turno = df_estados_1turno.sort_values(by = ['estado'], ascending = True)
df_estados_1turno.to_csv('/content/gdrive/MyDrive/TCC - Wallynson/df_estados_1turno_2020.csv')

In [ ]:
df_estados_1turno.to_csv('df.csv')